In [5]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')

# print(openai.Engine.list())

def print_lines(text, max_line_length=120):
    lines = []
    line = ""
    for word in text.split():
        if len(line + word) > max_line_length:
            lines.append(line)
            line = ""
        line += word + " "
    lines.append(line)

    for line in lines:
        print(line)

class ChatEngine:
    model = ""
    temp = 0

    def __init__(self, model="gpt-3.5-turbo", temp=0):
        self.model = model
        self.temp = temp

class Chat:
    messages: list = []
    engine: ChatEngine

    def __init__(self, engine, system_message, user_messages = []):
        self.engine = engine
        self.messages.append({"role": "system", "content": system_message})
        self.messages.extend([{"role": "user", "content": message} for message in user_messages])

    def get_answer(self, user_message):
        self.messages.append({"role": "user", "content": user_message})
        response = openai.ChatCompletion.create(
            model=self.engine.model,
            messages=self.messages,
            temperature=self.engine.temp,
        )
        answer = response.choices[0].message["content"]
        self.messages.append({"role": "assistant", "content": answer})
        return answer


In [ ]:
import json

continue_term_prompt = """You are a terms helper robot \
your task is to guess, based on the list of terms, \
which one term, not in the list, \
is the most relevant to already given terms."""

def load_global_static_data() -> dict:
    with open("../../lib/StaticDataStorage/data/Global.json", "r") as f:
        globalJson = json.load(f)
    
    return globalJson

def get_terms(globalJson: dict) -> list:
    ret = []

    for item in globalJson['terms']:
        termDef = item['termDef']
        term = termDef.split(" - ")[0]
        ret.append(term)

    return ret

def predict_next_term():
    engine = ChatEngine()
    chat = Chat(engine, continue_term_prompt)
    globalJson = load_global_static_data()
    terms = get_terms(globalJson)
    terms_str = ", ".join(terms)
    answer = chat.get_answer(terms_str)
    print_lines(answer)

predict_next_term()
